<a href="https://colab.research.google.com/github/Untick/Selector_ObjDet/blob/Roman-Penzov-folder/%D0%A1%D1%82%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B0_%D0%A0%D0%BE%D0%BC%D0%B0%D0%BD_%D0%9F%D0%B5%D0%BD%D0%B7%D0%BE%D0%B2_%D0%A8%D0%90%D0%93_4_05_06_23_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ШАГ 4.**
Мысли нарастающим итогом:
1.   Как и на шаге 1 для первоначального решения задачи не стал вычищать базу, а оставил только файлы формата jpg и jpeg. Некоторые папки (виды одежды) удалил целиком. А уже на заключительных шагах буду использовать "исправленную/актуализированную" базу.
2.   Также хочу пробовать следующий алгоритм:
  *   сначала первая нейронка должна определить категорию одежды, к которой относится"картинка клиента". И почему бы для этого не использовать для первого варианта обычные сверточные нейронные сети. Иначе возможно попробовать вариант проверки на выброс (ШАГ_4).
  *   также отдельно сделать "эталонную" политру по цветам, на основании которой сортировать изображения
  *   второй вид нейронки - это вариационный автокодировщик, который надо реализовать для каждой категории отдельно. Почему то думается, что так проще и эффективнее.
3. Написать функцию подбора схожих изображений на основе encoder.predict, NearestNeighbors (ШАГ_3)


In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, Flatten, Reshape, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import zipfile

In [2]:
# Работа с массивами

from PIL import ImageEnhance

# Основа для создания последовательной модели
from tensorflow.keras.models import Sequential

# Основные слои
from tensorflow.keras.layers import MaxPooling2D, Dropout, BatchNormalization

# Оптимизатор
from tensorflow.keras.optimizers import Adam, Adagrad

# Матрица ошибок классификатора
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Подключение модуля для загрузки данных из облака
import gdown

# Для работы с файлами
import gc

# Рисование графиков в ячейках Colab
%matplotlib inline

In [3]:
# Подключение к Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Путь к архивному файлу
file_path = '/content/gdrive/MyDrive/test_230501.zip'

# Путь к директории, в которую нужно разархивировать файл
extract_path = '/content/selector'

# Разархивирование файла
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [5]:
data_dir = '/content/selector/test_230501/'                  # Путь к базе
os.listdir(data_dir)
batch_size= 32
image_size= (256, 256)

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,    # Устанавливаем разделение на обучающую и проверочную выборки
  subset="training",       # Эта выборка обучающая
  seed=123,                # закрепляем рандом для сравнения сетей
  image_size= image_size,
  batch_size= batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",     # Эта выборка проверочная
  seed=123,
  image_size= image_size,
  batch_size= batch_size)

Found 7650 files belonging to 9 classes.
Using 6120 files for training.
Found 7650 files belonging to 9 classes.
Using 1530 files for validation.


In [7]:
class_names = train_ds.class_names # Определяем имена классов
print(class_names)

['dress  business', 'dress casual', 'dress homemade', 'dress solemn', 'shirt men', 'shirt women', 'sportswear women', 't-shirt men', 't-shirt women']


In [8]:
AUTOTUNE = tf.data.AUTOTUNE    # предварительная подгрузка датасета для ускорения и стабилизации

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE) 
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
# Аугментация в виде слоя для модели

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomZoom((-0.05,0.05)), # произвольное увеличение и уменьшение на 10%
  tf.keras.layers.experimental.preprocessing.RandomContrast(0.15)])    # изменение контраста изображений 

In [10]:
# Слой подготовки данных для vgg16(у каждой модели свой)
preprocess_input = tf.keras.applications.vgg16.preprocess_input
gc.collect()

# Создание модели
image_shape = image_size + (3,)
base_model = tf.keras.applications.vgg16.VGG16(input_shape=image_shape,
                         include_top=False,
                         weights='imagenet')

# base_model.summary()
global_average_layer = tf.keras.layers.GlobalAveragePooling2D() # Глобал пулинг
prediction_layer = tf.keras.layers.Dense(len(class_names), activation='softmax') # По количеству классов = 3 класса

# собираем полную модель 
inputs = tf.keras.Input(shape=(image_shape))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

58889256/58889256 [==============================] - 4s 0us/step


In [11]:
# компилируем

base_learning_rate = 0.00001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
initial_epochs = 60

In [12]:
# обучаем

hVGG16 = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)

Epoch 1/60
 64/192 [=========>....................] - ETA: 1:10 - loss: 2.4573 - accuracy: 0.2920

InvalidArgumentError: ignored